In [8]:
import pandas as pd
import json, os
from tqdm import tqdm

The following table represents URLs parsed from https://www.proteinatlas.org/download/proteinatlas.xml.gz, merged against https://www.proteinatlas.org/download/normal_tissue.tsv.zip and https://www.proteinatlas.org/download/rna_tissue_consensus.tsv.zip.

In [7]:
df = pd.read_csv('./data/hpa_v21_kidney.csv',index_col=0)
df

,Batch,Slide,Well,Tissue,Gene name,Gene,UniProt,Antibody,nTPM,Staining,Sex,Age,Patient,URL,Level,Reliability
Image,,,,,,,,,,,,,,,,
135_A_9_5,1,135,A_9_5,kidney,AR,ENSG00000169083,P10275,CAB000001,3.6,not detected,Male,61.0,443.0,http://images.proteinatlas.org/1/135_A_9_5.jpg,Medium,Enhanced
135_A_7_5,1,135,A_7_5,kidney,AR,ENSG00000169083,P10275,CAB000001,3.6,not detected,Female,52.0,1263.0,http://images.proteinatlas.org/1/135_A_7_5.jpg,Medium,Enhanced
135_A_8_5,1,135,A_8_5,kidney,AR,ENSG00000169083,P10275,CAB000001,3.6,not detected,Male,36.0,924.0,http://images.proteinatlas.org/1/135_A_8_5.jpg,Medium,Enhanced
1628_A_9_5,10,1628,A_9_5,kidney,CD3E,ENSG00000198851,P07766,CAB000010,3.4,not detected,Male,61.0,443.0,http://images.proteinatlas.org/10/1628_A_9_5.jpg,Not detected,Enhanced
1628_A_7_5,10,1628,A_7_5,kidney,CD3E,ENSG00000198851,P07766,CAB000010,3.4,not detected,Female,52.0,1263.0,http://images.proteinatlas.org/10/1628_A_7_5.jpg,Not detected,Enhanced
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103368_A_9_5,46031,103368,A_9_5,kidney,TUBA8,ENSG00000183785,Q9NY65,CAB046031,2.3,medium,Female,41.0,2530.0,http://images.proteinatlas.org/46031/103368_A_...,Medium,Approved
113569_A_9_5,49574,113569,A_9_5,kidney,ANKRD53,ENSG00000144031,Q8N9V6,HPA049574,0.8,high,Female,41.0,2530.0,http://images.proteinatlas.org/49574/113569_A_...,Not detected,Approved
124638_A_9_5,54976,124638,A_9_5,kidney,SLC29A3,ENSG00000198246,Q9BZD2,HPA054976,2.5,medium,Male,2.0,2887.0,http://images.proteinatlas.org/54976/124638_A_...,Medium,Approved


Here we download each image, pad or crop to a multiple of 512x512, downscale to 512x512, and cast to 8-bit PNG. We also associate with each a JSON file containing its metadata from the above table.

In [21]:
TISSUE = 'kidney'

for _, row in tqdm(df.reset_index().iterrows(),position=0):
    image = row['Image']
    batch = row['Batch']
    url = row['URL']
    path = f'./data/images/{TISSUE}/{batch}'
    !mkdir -p {path}
    with open(f'{path}/{image}.json','w') as f:
        json.dump(row.to_dict(), f)
    !curl -so {path}/{image}.jpg "{url}"'
    !mogrify -extent 3072x3072 -gravity center -scale 512x512 -format png8 {path}/{image}.jpg
    !rm {path}/{image}.jpg
    !mv {path}/{image}.png8 {path}/{image}.png

66138it [00:12, 5198.50it/s]
